In [ ]:
import common
from common import *

In [ ]:
#wf=load_node(125929)
#wf=load_node(128251)
wf=load_node(147770)
print(wf.label)
print(wf.description)
#wf2=load_node(129045)
#print(wf2.label)
#print(wf2.description)

In [ ]:
print_cp_with_traj(wf)

In [ ]:
res,axs,figs,n=force_ratio_histogram(wf)

In [ ]:
#res2,axs2,figs2,n2=force_ratio_histogram(wf2)

In [ ]:
fig,ax,axs=plot_force_ratio(res)
#fig2,ax2,axs2=plot_force_ratio(res2,fig=fig,ax=ax)
#ax2.grid()
ax.legend().set_visible(False)

In [ ]:
fig,ax,axs=plot_force_ratio(res,fig=fig2,ax=ax2)
# mu=np.array([40,90,160])
# H_c=1.0
# N_c=8.0-3*H_c
# ax.plot(mu,(1.66e-27*1.0-H_c*9.11e-31*mu/2)/1.66e-27*1.0,label='H dummy',c='k')
# ax.plot(mu,(1.66e-27*1-N_c*9.11e-31*mu/2)/(1.66e-27*1),label='N dummy',c='r')
# ax.legend(loc=3)

In [ ]:
#full_t=wf.outputs.full_traj
#full_t.export('ammoniaca_esplorazione_diagramma_fase/solid.axsf',fileformat='xsf')

In [ ]:
#full_t._repository._get_base_folder().abspath

In [ ]:

#traj=load_node(126884) # superionic
#traj=load_node(126746) # superionic
#traj=load_node(126722) # maybe plastic -- no, it is superionic
#traj=load_node(126650) # maybe plastic -- no, it is superioni (very weak)
traj=wf.outputs.nve_prod_traj


In [ ]:
wf.description

In [ ]:
wf.outputs.emass

In [ ]:
pickle_dump_name=f'traj_{traj.pk}.pickle'

In [ ]:
print('traj pk = {}'.format(traj.pk))
t=traj.get_array('times')

natoms=traj.numsites
#conversion factor from eV to K
k_b=8.617333262145e-5 #eV/K
eV_to_K=2/(3*k_b*natoms)

print('traj arraynames = {}'.format(traj.get_arraynames()))

cmot=traj.get_array('energy_constant_motion')
temp=traj.get_array('ionic_temperature')
press=traj.get_array('pressure')
T_mean=temp.mean()
press_mean=press.mean()
DT_FS=(t[1]-t[0])*1e3
print('DT_FS = {}fs, T = {:.4e}K, P = {:.4e}GPa'.format(DT_FS,T_mean,press_mean))
cell_0=traj.get_array('cells')[0]
volume=np.linalg.det(cell_0)
print('cell = {}'.format(cell_0))
print('cell_volume = {} A^3'.format(volume))

In [ ]:
plt_key(traj,'electronic_kinetic_energy',eV_to_K,ylabel='K')
plt_key(traj,'energy_constant_motion')
plt_key(traj,'ionic_temperature',ylabel='K')
plt_key(traj,'pressure',ylabel='GPa')

In [ ]:
np.corrcoef(cmot-cmot.mean(),temp-temp.mean())

In [ ]:
ekinc_const_motion_analysis(traj)

In [ ]:


#write_cp_traj.write_cp_trajectory('test_traj_400K_25GPa_100ps/solid_traj',
#                                  traj.get_array('times'),
#                                  traj.get_array('positions'),
#                                  traj.get_array('velocities'),
#                                  traj.get_array('cells'))

In [ ]:
traj2=traj

atraj2,atraj2_unw=get_analisi_traj_from_aiida(traj2)
resB=atomic_density(atraj2,traj2.get_array('cells')[0])
resgnB=fft_density(resB)
resgn2B=fft_density(resB,np.array([0,1]))

In [ ]:
plotG=k3d.plot()
plotG+=k3d.volume(np.array(resgn2B,dtype='float32'))
#plotG+=k3d.volume(np.array(resgn2,dtype='float32'))
plotG.display()

In [ ]:
plotG=k3d.plot()
plotG+=k3d.volume(np.array(resgnB,dtype='float32'))
#plotG+=k3d.volume(np.array(resgn2,dtype='float32'))
plotG.display()

In [ ]:
plotG=k3d.plot()
plotG+=k3d.volume(scipy.ndimage.gaussian_filter(resB[0],1.2,output='float32'))
plotG+=k3d.volume(scipy.ndimage.gaussian_filter(resB[1],1.2,output='float32'))
#plotG+=k3d.volume(np.array(resgn2,dtype='float32'))
plotG.display()

In [ ]:
atraj,atraj_unw=get_analisi_traj_from_aiida(traj)

In [ ]:
res=atomic_density(atraj,cell_0)

In [ ]:
resgn=fft_density(res)
resgn2=fft_density(res,np.array([0,1]))

In [ ]:
@interact(x=widgets.IntSlider(min=0, max=resgn.shape[0]-1, step=1, value=50))
def f(x):
    plt.figure(dpi=200)
    plt.imshow(resgn[x],
               norm=pltc.LogNorm(vmin=0.0001,vmax=resgn.max()),
               extent=[-resgn.shape[1]/2., resgn.shape[1]/2., -resgn.shape[0]/2., resgn.shape[0]/2. ])

In [ ]:
plotG=k3d.plot()
plotG+=k3d.volume(np.array(resgn2,dtype='float32'))
plotG.display()

In [ ]:
res_g=scipy.ndimage.gaussian_filter(res[0],1.2,output='float32')
res_g1=scipy.ndimage.gaussian_filter(res[1],1.2,output='float32')

_, g1_sites = get_local_maximum_idx_s(res_g1,res_g1.max()*0.5 ,minimum_distance=0.5,l=8.5)
_, g0_sites = get_local_maximum_idx_s(res_g,res_g.max()*0.5,minimum_distance=0.5,l=8.5)

print(len(g1_sites),len(g0_sites))

In [ ]:
density_field(res_g,res_g1)

In [ ]:
plotN=k3d.plot()
points = k3d.points(g0_sites,point_size=0.5)
points1 = k3d.points(g1_sites,point_size=0.1)
plotN+=points
plotN+=points1
plotN.display()

In [ ]:
import pymatgen as pmg

In [ ]:
#abc=[ traj.get_array('cells')[0,i,i] for i in range(3)]
lattice=pmg.core.lattice.Lattice(traj.get_array('cells')[0])

In [ ]:
N_structure = pmg.core.structure.Structure(lattice=lattice,
                                           species=['N']*len(g0_sites),
                                           coords=g0_sites,
                                           coords_are_cartesian=False)

In [ ]:
N_structure_primitive=N_structure.get_primitive_structure()

In [ ]:
N_structure.get_primitive_structure()

In [ ]:
N_structure_primitive.make_supercell([3,3,3])
N_structure_primitive.cart_coords

In [ ]:
plotN2=k3d.plot()
points2 = k3d.points(N_structure_primitive.cart_coords,point_size=0.1)
plotN2+=points2
plotN2.display()

In [ ]:
analisi = pickle_or_unpickle(pickle_dump_name)
if analisi is None:
    analisi=analyze(atraj,atraj_unw,0,atraj.getNtimesteps(),tskip_msd=10,tskip_sh=40)
    pickle_or_unpickle(pickle_dump_name,analisi = analisi)
msd,gofr,sh=analisi

In [ ]:
plot_gofr(0.5,3.8,gofr)

In [ ]:
ax,axins,fitres=plot_sh(0.7,3.5,t,sh,0,1,0,scale_exp_coeff=0.2,log=False)
ax.set_ylim([0e-2,1.0])

In [ ]:
plot_msd(t,msd,0)

# transport properties and cepstral analysis

In [ ]:
import thermocepstrum as tc

In [ ]:
jfile = tc.i_o.TableFile(current_file, group_vectors=True)

In [ ]:
jfile.read_datalines(select_ckeys=['j', 'je', 'jn','jh'])

In [ ]:
j = tc.HeatCurrent([jfile.data['j'],jfile.data['je'],jfile.data['jh']],
                   UNITS='qepw', DT_FS=DT_FS, TEMPERATURE=T_mean, VOLUME=volume)

In [ ]:

f, ax = plt.subplots(2, sharex=True)
ax[0].plot(j.timeseries()/1000., j.traj);
ax[1].plot(j.timeseries()/1000., j.otherMD[0].traj);
plt.xlim([50, 50.1])
plt.xlabel(r'$t$ [ps]')
ax[0].set_ylabel(r'$J^0$ [eV A/ps]');
ax[1].set_ylabel(r'$J^1$ [A/ps]');

In [ ]:
ax = j.plot_periodogram(PSD_FILTER_W=0.4, kappa_units=True, label=r'$\bar{\mathcal{S}}^0_k$')
print(j.Nyquist_f_THz)

# compare with the spectrum of the energy flux
jen = tc.HeatCurrent(jfile.data['j'],
                     UNITS='qepw', DT_FS=DT_FS, TEMPERATURE=T_mean, VOLUME=volume)
ax = jen.plot_periodogram(axes=ax, PSD_FILTER_W=0.4, kappa_units=True, label=r'$\mathcal{S}^0_k$')
#plt.xlim([0, 20])
#ax[0].set_ylim([0, 0.8]);
#ax[1].set_ylim([7, 18]);
ax[0].legend(); ax[1].legend();


In [ ]:
ax = j.plot_periodogram(PSD_FILTER_W=0.4, kappa_units=True, label=r'$\bar{\mathcal{S}}^0_k$')
print(j.Nyquist_f_THz)
ax = jen.plot_periodogram(axes=ax, PSD_FILTER_W=0.4, kappa_units=True, label=r'$\mathcal{S}^0_k$')
plt.xlim([0, 40])
ax[0].set_ylim([0, 10]);
#ax[1].set_ylim([7, 18]);
ax[0].legend(); ax[1].legend();

In [ ]:
FSTAR_THZ = 40.0
jf, ax = j.resample(fstar_THz=FSTAR_THZ, plot=True, freq_units='thz')
plt.xlim([0, 40])
#ax[1].set_ylim([7, 18]);

In [ ]:
jf.cepstral_analysis()

In [ ]:
# Cepstral Coefficients
print('c_k = ', jf.dct.logpsdK)

ax = jf.plot_ck()
ax.set_xlim([0, 50])
ax.set_ylim([-0.2, 1.0])
ax.grid();

In [ ]:
# AIC function
f = plt.figure()
plt.plot(jf.dct.aic, '.-')
plt.xlim([0, 50])
#plt.ylim([1400, 1500]);

print('K of AIC_min = {:d}'.format(jf.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf.dct.aic_min))

In [ ]:

# L_0 as a function of cutoff K
ax = jf.plot_L0_Pstar()
ax.set_xlim([0, 50])
#ax.set_ylim([14.75, 16.5]);

print('K of AIC_min = {:d}'.format(jf.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf.dct.aic_min))

In [ ]:
# kappa as a function of cutoff K
ax = jf.plot_kappa_Pstar()
ax.set_xlim([0, 50])
ax.set_ylim([0, 6.0]);

print('K of AIC_min = {:d}'.format(jf.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf.dct.aic_min))

In [ ]:

# filtered log-PSD
ax = j.plot_periodogram(0.5, kappa_units=True)
ax = jf.plot_periodogram(0.5, axes=ax, kappa_units=True)
ax = jf.plot_cepstral_spectrum(axes=ax, kappa_units=True)
ax[0].axvline(x = jf.Nyquist_f_THz, ls='--', c='r')
ax[1].axvline(x = jf.Nyquist_f_THz, ls='--', c='r')
plt.xlim([0, 20])
ax[0].set_ylim([0, 10.8]);
#ax[1].set_ylim([7, 18]);
ax[0].legend(['original', 'resampled', 'cepstrum-filtered'])
ax[1].legend(['original', 'resampled', 'cepstrum-filtered']);

# electric conductivity
Note: in the header the label jh and jn are inverted: the label jh is the Nitrogen current while jn labels Hydrogen current.

In [ ]:
j_el = tc.ElectricCurrent([-jfile.data['je']+32*5*jfile.data['jh']+96*jfile.data['jn']],
                   UNITS='qepw', DT_FS=DT_FS, TEMPERATURE=T_mean, VOLUME=volume)


f, ax = plt.subplots(2, sharex=True)
ax[0].plot(j_el.timeseries()/1000., j_el.traj);
plt.xlim([50, 50.1])
plt.xlabel(r'$t$ [ps]')
ax[0].set_ylabel(r'$J^0$ [eV A/ps]');

ax = j_el.plot_periodogram(PSD_FILTER_W=0.4, kappa_units=True, label=r'$\bar{\mathcal{S}}^0_k$')
print(j.Nyquist_f_THz)





FSTAR_THZ = 40.0
jf_el, ax = j_el.resample(fstar_THz=FSTAR_THZ, plot=True, freq_units='thz')
plt.xlim([0, 40])
#ax[1].set_ylim([7, 18]);

jf_el.cepstral_analysis()

# Cepstral Coefficients
print('c_k = ', jf_el.dct.logpsdK)

ax = jf_el.plot_ck()
ax.set_xlim([0, 50])
#ax.set_ylim([-0.2, 1.0])
ax.grid();

# AIC function
f = plt.figure()
plt.plot(jf_el.dct.aic, '.-')
plt.xlim([0, 50])
#plt.ylim([1400, 1500]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))


# L_0 as a function of cutoff K
ax = jf_el.plot_L0_Pstar()
ax.set_xlim([0, 200])
#ax.set_ylim([14.75, 16.5]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))

# kappa as a function of cutoff K
ax = jf_el.plot_kappa_Pstar()
ax.set_xlim([0, 200])
#ax.set_ylim([0, 6.0]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))


# filtered log-PSD
ax = j_el.plot_periodogram(0.5, kappa_units=True)
ax = jf_el.plot_periodogram(0.5, axes=ax, kappa_units=True)
ax = jf_el.plot_cepstral_spectrum(axes=ax, kappa_units=True)
ax[0].axvline(x = jf_el.Nyquist_f_THz, ls='--', c='r')
ax[1].axvline(x = jf_el.Nyquist_f_THz, ls='--', c='r')
plt.xlim([0, 20])
#ax[0].set_ylim([0, 10.8]);
#ax[1].set_ylim([7, 18]);
ax[0].legend(['original', 'resampled', 'cepstrum-filtered'])
ax[1].legend(['original', 'resampled', 'cepstrum-filtered']);

j_el = tc.ElectricCurrent([32*3*jfile.data['jh']-96*jfile.data['jn']],
                   UNITS='qepw', DT_FS=DT_FS, TEMPERATURE=T_mean, VOLUME=volume)


f, ax = plt.subplots(2, sharex=True)
ax[0].plot(j_el.timeseries()/1000., j_el.traj);
plt.xlim([50, 50.1])
plt.xlabel(r'$t$ [ps]')
ax[0].set_ylabel(r'$J^0$ [eV A/ps]');

ax = j_el.plot_periodogram(PSD_FILTER_W=0.4, kappa_units=True, label=r'$\bar{\mathcal{S}}^0_k$')
print(j.Nyquist_f_THz)





FSTAR_THZ = 40.0
jf_el, ax = j_el.resample(fstar_THz=FSTAR_THZ, plot=True, freq_units='thz')
plt.xlim([0, 40])
#ax[1].set_ylim([7, 18]);

jf_el.cepstral_analysis()

# Cepstral Coefficients
print('c_k = ', jf_el.dct.logpsdK)

ax = jf_el.plot_ck()
ax.set_xlim([0, 50])
#ax.set_ylim([-0.2, 1.0])
ax.grid();

# AIC function
f = plt.figure()
plt.plot(jf_el.dct.aic, '.-')
plt.xlim([0, 50])
#plt.ylim([1400, 1500]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))


# L_0 as a function of cutoff K
ax = jf_el.plot_L0_Pstar()
ax.set_xlim([0, 200])
#ax.set_ylim([14.75, 16.5]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))

# kappa as a function of cutoff K
ax = jf_el.plot_kappa_Pstar()
ax.set_xlim([0, 200])
#ax.set_ylim([0, 6.0]);

print('K of AIC_min = {:d}'.format(jf_el.dct.aic_Kmin))
print('AIC_min = {:f}'.format(jf_el.dct.aic_min))


# filtered log-PSD
ax = j_el.plot_periodogram(0.5, kappa_units=True)
ax = jf_el.plot_periodogram(0.5, axes=ax, kappa_units=True)
ax = jf_el.plot_cepstral_spectrum(axes=ax, kappa_units=True)
ax[0].axvline(x = jf_el.Nyquist_f_THz, ls='--', c='r')
ax[1].axvline(x = jf_el.Nyquist_f_THz, ls='--', c='r')
plt.xlim([0, 20])
#ax[0].set_ylim([0, 10.8]);
#ax[1].set_ylim([7, 18]);
ax[0].legend(['original', 'resampled', 'cepstrum-filtered'])
ax[1].legend(['original', 'resampled', 'cepstrum-filtered']);

In [ ]:
freq = load_node(145978)


In [ ]:
bugged_traj=freq.get_incoming().all_nodes()[0].get_incoming().all_nodes()[0]

In [ ]:
get_maximum_frequency_vdos(bugged_traj)

In [ ]:
bugged_traj.get_array('times')

In [ ]:
plt.plot(np.fft.fftfreq(bugged_traj.get_array('velocities').shape[0],bugged_traj.get_array('times')[-1]-bugged_traj.get_array('times')[-2]),
         np.abs(np.fft.fft(bugged_traj.get_array('velocities'),axis=0)).mean(axis=1).mean(axis=1))

In [ ]:
if  hasattr(bugged_traj,'pippo'):
    print('ciao')

In [ ]:
bugged_traj.get_array('times').shape[0]